In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
from pandas.tseries.offsets import BDay
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from dateutil.parser import parse
from statsmodels.tsa.stattools import adfuller, kpss
from pandas.plotting import autocorrelation_plot
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

pd.set_option("display.max_rows", None, "display.max_columns", None)

#df3.toPandas().to_csv('./df3.csv', index=False)
#LoadCSV
df = pd.read_csv('./dfarimadrop.csv', sep=";")


In [2]:
df

,avg_temp_motor1
0,16.20
1,15.75
2,17.33
3,20.25
4,17.00
5,15.75
6,19.25
7,14.08
8,18.33
9,16.50


In [3]:
def hold(y, ratio=2 / 3, mode="rolling", it=1, window=10, increment=1):
    #ratio = test size
    #window = traingin size
    ALLITR = None
    VAL = None
    NSIZE = len(y)
    if mode == "incremental" or mode == "rolling":
        aux = window + increment * (it - 1)
        aux = min(aux, NSIZE)
        if mode == "rolling":
            iaux = max((aux - window + 1), 1)
        else:
            iaux = 1
        ALLTR = range(iaux, aux)
        end = aux + ratio
        end = min(end, NSIZE)
        iend = aux + 1
        if iend < end:
            TS = range(iend, end)
        else:
            TS = None
    return {"tr": ALLTR, "itr": ALLITR, "val": VAL, "ts": TS}

In [4]:
dfarimadrop = df

In [5]:
def calcIterationsRW(datasetSize, trainingSize, testSize, stepSize): 
    
    return int((datasetSize - (trainingSize + testSize))/ stepSize)

trainingSize = 1023
testSize = 15
ratio = testSize
stepSize = 200
#totalLenght = df.shape[0]
mode = "rolling"

iterations = calcIterationsRW(dfarimadrop.shape[0], trainingSize, testSize, stepSize)

#For only positive records
#iterations = calcIterationsRW(len(df['serial'].unique()), trainingSize, testSize, stepSize)

iterations

24

In [6]:
from pmdarima.arima import auto_arima
from tqdm import trange
from sklearn.metrics import mean_absolute_error

def runarima(data, iterations):
    max_value = data['avg_temp_motor1'].max()
    min_value = data['avg_temp_motor1'].min()
    denominator = max_value - min_value
    
    dfFinal = pd.DataFrame(columns = ['MAE', 'NMAE'])
    
    MAEVals = []
    NMAEVals = []
   
    for i in trange(iterations):

            holds = hold(data[['avg_temp_motor1']], ratio, mode, i+1, trainingSize, stepSize)
            print('holds')
            print("Iteration ", i+1 , "of ", iterations, " TR: ", holds['tr'] , "TS: ", holds['ts'])
            tr = data.iloc[holds['tr']].reset_index(drop=True)
            ts = data.iloc[holds['ts']].reset_index(drop=True)
            
            model =  auto_arima(tr[["avg_temp_motor1"]], start_p=0, d=1, start_q=0, 
                          max_p=5, max_d=5, max_q=5, start_P=0, 
                          D=1, start_Q=0, max_P=5, max_D=5,
                          max_Q=5, m=12, seasonal=True, 
                          error_action='warn',trace = True,
                          supress_warnings=True,stepwise = True,
                          random_state=20,n_fits = 20)
        #prediction = pd.DataFrame(model.predict(n_periods = len(test)), index=test.index)            
         #   p = model.predict(n_periods= len(testSize), index=text.index) 
            p = model.predict(n_periods=14)
            MAE = mean_absolute_error(ts.values.tolist(), p)
            NMAE = (MAE/denominator)*100 #o denominador é calculado com o maximo e minimo global
            
            print("Iteration ", i , "MAE ", MAE, "NMAE of ", NMAE)
            
            MAEVals.append(MAE)
            NMAEVals.append(NMAE)
        
            new = {'MAE': MAE, 'NMAE': NMAE}
            dfFinal = dfFinal.append(new, ignore_index=True)
            dfFinal = pd.DataFrame(dfFinal)
            
    return dfFinal

In [ ]:
dfResults = runarima(dfarimadrop,iterations)

  0%|          | 0/24 [00:00<?, ?it/s]

holds
Iteration  1 of  24  TR:  range(1, 1023) TS:  range(1024, 1038)
Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=5801.300, Time=0.08 sec
 ARIMA(1,1,0)(1,1,0)[12]             : AIC=5154.492, Time=0.40 sec
 ARIMA(0,1,1)(0,1,1)[12]             : AIC=inf, Time=2.97 sec
 ARIMA(1,1,0)(0,1,0)[12]             : AIC=5443.258, Time=0.10 sec
 ARIMA(1,1,0)(2,1,0)[12]             : AIC=5052.807, Time=1.04 sec
 ARIMA(1,1,0)(3,1,0)[12]             : AIC=4995.126, Time=2.94 sec
 ARIMA(1,1,0)(4,1,0)[12]             : AIC=4948.030, Time=5.83 sec
 ARIMA(1,1,0)(5,1,0)[12]             : AIC=4912.711, Time=10.49 sec
 ARIMA(1,1,0)(5,1,1)[12]             : AIC=inf, Time=65.29 sec
 ARIMA(1,1,0)(4,1,1)[12]             : AIC=inf, Time=36.64 sec
 ARIMA(0,1,0)(5,1,0)[12]             : AIC=5216.882, Time=8.92 sec
 ARIMA(2,1,0)(5,1,0)[12]             : AIC=4804.273, Time=12.74 sec
 ARIMA(2,1,0)(4,1,0)[12]             : AIC=4837.691, Time=7.95 sec
